In [11]:
!pip install boto3 pyarrow pandas

In [12]:
import boto3

ACCESS_KEY = "71e05046d84a44308f53c118b502cb6a"
SECRET_KEY = "da42757432504c92b9f2d21e5c6e5634"

s3 = boto3.client(
    service_name="s3",
    region_name="kr-central-2",
    endpoint_url="https://objectstorage.kr-central-2.kakaocloud.com",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
)


In [16]:
# Cell 1: Parquet 파일 키 목록 가져오기
import io
import random
import pyarrow.parquet as pq
import pandas as pd

bucket_name = "data-catalog-bucket"
prefix      = "data-catalog-dir/user_behavior_prediction_batch/"

# Prefix 아래 모든 .parquet 키 리스트
resp = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
keys = [
    obj["Key"]
    for obj in resp.get("Contents", [])
    if obj["Key"].endswith(".parquet")
]

if not keys:
    raise ValueError(f"No parquet files found under prefix {prefix!r}")


In [17]:
# Cell 2: 무작위 Parquet 파일 선택
random_key = random.choice(keys)


In [18]:
# Cell 3: 선택된 Parquet 파일 읽어서 DataFrame으로 변환·출력
obj = s3.get_object(Bucket=bucket_name, Key=random_key)
buf = io.BytesIO(obj["Body"].read())
table = pq.read_table(buf)
df = table.to_pandas()

df.head(10)


,session_id,user_id,gender,age,current_state,search_count,cart_item_count,page_depth,last_action_elapsed,next_state,timestamp,avg_response_time,current_product_id,session_duration,query_params,search_keyword,product_id
0,04bd2bd5-cd94-4900-b3ca-303210a178b1,None,None,NaN,/error,0,0,2,1.0,None,2025-07-30 20:31:52,0.012,NaN,1.0,None,None,None
1,04bd2bd5-cd94-4900-b3ca-303210a178b1,None,None,NaN,/products,0,0,1,0.0,/error,2025-07-30 20:31:51,0.012,NaN,0.0,None,None,None
2,08130a75-56ec-4f9f-bdfb-cff7f8ba4145,None,None,NaN,/add_user,0,0,1,0.0,/products,2025-07-30 13:43:53,0.020,NaN,0.0,None,None,None
3,08130a75-56ec-4f9f-bdfb-cff7f8ba4145,None,None,NaN,/login,2,0,5,3.0,/add_review,2025-07-30 13:43:59,0.020,NaN,6.0,None,None,None
4,08130a75-56ec-4f9f-bdfb-cff7f8ba4145,None,None,NaN,/products,0,0,2,1.0,/search,2025-07-30 13:43:54,0.020,NaN,1.0,None,None,None
5,08130a75-56ec-4f9f-bdfb-cff7f8ba4145,None,None,NaN,/search,1,0,3,1.0,/search,2025-07-30 13:43:55,0.020,NaN,2.0,None,Fashion,None
6,08130a75-56ec-4f9f-bdfb-cff7f8ba4145,None,None,NaN,/search,2,0,4,1.0,/login,2025-07-30 13:43:56,0.020,NaN,3.0,None,Bluetooth,None
7,08130a75-56ec-4f9f-bdfb-cff7f8ba4145,user_5e6d1b,F,34.0,/add_review,2,0,6,1.0,/logout,2025-07-30 13:44:00,0.020,123.0,7.0,None,None,123
8,08130a75-56ec-4f9f-bdfb-cff7f8ba4145,user_5e6d1b,F,34.0,/logout,2,0,7,2.0,None,2025-07-30 13:44:02,0.020,NaN,9.0,None,None,None
9,095db3a2-1d46-4a91-afaf-418a16fe3126,None,None,NaN,/add_user,0,0,1,0.0,/search,2025-07-31 01:05:47,0.021,NaN,0.0,None,None,None
